# Approval Workflow — Human-in-the-Loop for High-Risk Actions

SafeAI supports **human-in-the-loop approval gates** for high-risk actions. When a policy rule uses `action: require_approval`, the interceptor pauses execution and creates an approval request that must be explicitly approved or denied by a human reviewer before the action can proceed.

This notebook demonstrates:
- Triggering approval requests via policy-driven gates
- Listing, approving, and denying requests
- TTL-based expiry
- Automatic deduplication of identical requests

In [1]:
import tempfile
import re
from pathlib import Path

import yaml
from click.testing import CliRunner
from safeai.cli.init import init_command
from safeai import SafeAI

# Scaffold a fresh SafeAI project in a temp directory
tmp = tempfile.TemporaryDirectory()
work = Path(tmp.name)
CliRunner().invoke(init_command, ["--path", str(work)])

# --- Add a require_approval policy rule for destructive actions ---
policy_path = work / "policies" / "default.yaml"
policy_doc = yaml.safe_load(policy_path.read_text())
policy_doc["policies"].insert(0, {
    "name": "approval-gate-destructive",
    "boundary": ["action"],
    "priority": 5,
    "condition": {"data_tags": ["destructive"]},
    "action": "require_approval",
    "reason": "Destructive operations require human approval.",
})
policy_path.write_text(yaml.dump(policy_doc, sort_keys=False))

# --- Register a tool contract for delete_user ---
contract_doc = {
    "version": "v1alpha1",
    "contract": {
        "tool_name": "delete_user",
        "description": "Permanently deletes a user account.",
        "accepts": {"tags": ["destructive", "internal"], "fields": ["user_id"]},
        "emits": {"tags": ["internal"], "fields": ["status"]},
        "stores": {"fields": ["user_id"], "retention": "90d"},
        "side_effects": {
            "reversible": False,
            "requires_approval": True,
            "description": "User deletion is permanent and irreversible.",
        },
    },
}
contracts_dir = work / "contracts"
contracts_dir.mkdir(parents=True, exist_ok=True)
(contracts_dir / "delete_user.yaml").write_text(yaml.dump(contract_doc, sort_keys=False))

# --- Register an agent identity for bot-1 with delete_user binding ---
identity_doc = {
    "version": "v1alpha1",
    "agent": {
        "agent_id": "bot-1",
        "description": "Automation bot with destructive tool access.",
        "tools": ["delete_user"],
        "clearance_tags": ["destructive", "internal"],
    },
}
agents_dir = work / "agents"
agents_dir.mkdir(parents=True, exist_ok=True)
(agents_dir / "bot-1.yaml").write_text(yaml.dump(identity_doc, sort_keys=False))

# Load SafeAI with the updated config
ai = SafeAI.from_config(work / "safeai.yaml")
print("SafeAI loaded with approval-gate policy, delete_user contract, and bot-1 identity.")

SafeAI loaded with approval-gate policy, delete_user contract, and bot-1 identity.


## 1. Trigger an approval request

When an agent calls a tool tagged `destructive`, the policy engine matches the `approval-gate-destructive` rule and returns a `require_approval` decision instead of allowing or blocking. The interceptor automatically creates an approval request and embeds the request ID in the reason string.

In [2]:
result = ai.intercept_tool_request(
    tool_name="delete_user",
    parameters={"user_id": "123"},
    data_tags=["destructive"],
    agent_id="bot-1",
)

print(f"Decision : {result.decision.action}")
print(f"Policy   : {result.decision.policy_name}")
print(f"Reason   : {result.decision.reason}")

# Extract the approval request_id from the reason string
match = re.search(r"(apr_[a-f0-9]+)", result.decision.reason)
request_id = match.group(1) if match else None
print(f"\nExtracted request_id: {request_id}")

Decision : require_approval
Policy   : approval-gate-destructive
Reason   : approval required (apr_1cdf926483f3)

Extracted request_id: apr_1cdf926483f3


## 2. List pending requests

The approval manager tracks all requests. You can filter by status, agent, or tool.

In [3]:
pending = ai.list_approval_requests(status="pending")
print(f"Pending requests: {len(pending)}\n")

for req in pending:
    print(f"  request_id  : {req.request_id}")
    print(f"  status      : {req.status}")
    print(f"  agent_id    : {req.agent_id}")
    print(f"  tool_name   : {req.tool_name}")
    print(f"  data_tags   : {req.data_tags}")
    print(f"  reason      : {req.reason}")
    print(f"  policy_name : {req.policy_name}")
    print(f"  requested_at: {req.requested_at}")
    print(f"  expires_at  : {req.expires_at}")
    print()

Pending requests: 1

  request_id  : apr_1cdf926483f3
  status      : pending
  agent_id    : bot-1
  tool_name   : delete_user
  data_tags   : ['destructive']
  reason      : Destructive operations require human approval.
  policy_name : approval-gate-destructive
  requested_at: 2026-02-21 11:16:07.296028+00:00
  expires_at  : 2026-02-21 11:46:07.296028+00:00



## 3. Approve a request

A human reviewer (or automation with the right role) calls `approve_request` with an approver ID and optional note. The request transitions from `pending` to `approved`.

In [4]:
success = ai.approve_request(request_id, approver_id="admin", note="Verified safe")
print(f"Approve returned: {success}")

# Verify the status changed
all_requests = ai.list_approval_requests()
approved_req = next(r for r in all_requests if r.request_id == request_id)
print(f"\nUpdated status   : {approved_req.status}")
print(f"Approver         : {approved_req.approver_id}")
print(f"Decision note    : {approved_req.decision_note}")
print(f"Decided at       : {approved_req.decided_at}")

Approve returned: True

Updated status   : approved
Approver         : admin
Decision note    : Verified safe
Decided at       : 2026-02-21 11:16:07.302944+00:00


## 4. Denied request

A reviewer can also deny a request. Once denied, the tool call is permanently blocked for that request.

In [5]:
# Trigger a new approval request (different parameters to avoid deduplication)
result2 = ai.intercept_tool_request(
    tool_name="delete_user",
    parameters={"user_id": "456"},
    data_tags=["destructive"],
    agent_id="bot-1",
)

match2 = re.search(r"(apr_[a-f0-9]+)", result2.decision.reason)
request_id_2 = match2.group(1) if match2 else None
print(f"New request_id: {request_id_2}")
print(f"Status        : {result2.decision.action}")

# Deny this request
denied = ai.deny_request(request_id_2, approver_id="admin", note="Not authorized for this user")
print(f"\nDeny returned: {denied}")

# Confirm denied status
denied_req = next(r for r in ai.list_approval_requests() if r.request_id == request_id_2)
print(f"Status        : {denied_req.status}")
print(f"Decision note : {denied_req.decision_note}")

New request_id: apr_c83364bca585
Status        : require_approval

Deny returned: True
Status        : denied
Decision note : Not authorized for this user


## 5. TTL expiry

Every approval request has an `expires_at` timestamp based on the configured TTL (default `30m` from `safeai.yaml`). If no decision is made before the TTL elapses, the request status transitions to `expired` and the action is not allowed.

In [6]:
# Trigger another request and inspect its TTL window
result3 = ai.intercept_tool_request(
    tool_name="delete_user",
    parameters={"user_id": "789"},
    data_tags=["destructive"],
    agent_id="bot-1",
)

match3 = re.search(r"(apr_[a-f0-9]+)", result3.decision.reason)
request_id_3 = match3.group(1) if match3 else None

ttl_req = next(r for r in ai.list_approval_requests() if r.request_id == request_id_3)
print(f"Request ID    : {ttl_req.request_id}")
print(f"Status        : {ttl_req.status}")
print(f"Requested at  : {ttl_req.requested_at}")
print(f"Expires at    : {ttl_req.expires_at}")

delta = ttl_req.expires_at - ttl_req.requested_at
print(f"\nTTL window    : {delta} ({int(delta.total_seconds())} seconds)")
print("\nIf no approve/deny happens within this window, the request becomes 'expired'.")
print("Expired requests cannot be approved or denied — a new request must be created.")

Request ID    : apr_88af34718947
Status        : pending
Requested at  : 2026-02-21 11:16:07.310542+00:00
Expires at    : 2026-02-21 11:46:07.310542+00:00

TTL window    : 0:30:00 (1800 seconds)

If no approve/deny happens within this window, the request becomes 'expired'.
Expired requests cannot be approved or denied — a new request must be created.


## 6. Deduplication

When the same agent triggers the same tool with the same tags and parameters, SafeAI reuses the existing pending request instead of creating a duplicate. This prevents flooding the approval queue with redundant entries.

In [7]:
# Trigger the exact same call twice — same agent, tool, tags, and parameter keys
call_a = ai.intercept_tool_request(
    tool_name="delete_user",
    parameters={"user_id": "999"},
    data_tags=["destructive"],
    agent_id="bot-1",
)

call_b = ai.intercept_tool_request(
    tool_name="delete_user",
    parameters={"user_id": "999"},
    data_tags=["destructive"],
    agent_id="bot-1",
)

id_a = re.search(r"(apr_[a-f0-9]+)", call_a.decision.reason).group(1)
id_b = re.search(r"(apr_[a-f0-9]+)", call_b.decision.reason).group(1)

print(f"First  call request_id: {id_a}")
print(f"Second call request_id: {id_b}")
print(f"\nSame request reused? {id_a == id_b}")
print("\nDeduplication is keyed on: agent_id + tool_name + session_id + source + data_tags + parameter_keys")

First  call request_id: apr_88af34718947
Second call request_id: apr_88af34718947

Same request reused? True

Deduplication is keyed on: agent_id + tool_name + session_id + source + data_tags + parameter_keys


In [8]:
# Cleanup
tmp.cleanup()
print("Temporary project cleaned up.")

Temporary project cleaned up.


---

## Summary

SafeAI's approval workflow provides a policy-driven human-in-the-loop gate for high-risk agent actions:

| Feature | How it works |
|---|---|
| **Trigger** | Policy rules with `action: require_approval` matched by data tags |
| **Approval** | `approve_request(request_id, approver_id=..., note=...)` |
| **Denial** | `deny_request(request_id, approver_id=..., note=...)` |
| **TTL expiry** | Requests auto-expire after the configured TTL (default 30m) |
| **Deduplication** | Identical pending requests are coalesced by a composite dedupe key |
| **Audit trail** | Every decision is logged via the audit system |

All approval state is persisted to the `approvals.file_path` configured in `safeai.yaml`, making it safe across process restarts.